In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import overview data
overview = pd.read_excel("debate_data/data_overviews/Scraping_Xls.xlsx")
overview.columns = ["date","year","type","title","moderator"]

In [3]:
overview.head()

,date,year,type,title,moderator
0,"1960\nOctober 21, 1960\n",1960,General,\nPresidential Debate in New York,"['Howe','Edwards','Vanocur']"
1,"1976\nOctober 22, 1976\n",1976,General,\nPresidential Campaign Debate in Williamsburg...,"['Walters','Nelson','Kraft','Maynard']"
2,"1988\nOctober 13, 1988\n",1988,General,\nPresidential Debate in Los Angeles,"['SHAW','Ann Compton', 'Margaret Warner', 'Mit..."
3,"1992\nOctober 19, 1992\n",1992,General,"\nPresidential Debate in East Lansing, Michigan",['Lehrer']
4,"1996\nOctober 16, 1996\n",1996,General,\nPresidential Debate in San Diego,['Lehrer']


In [4]:
# remove undercard debates

count = 0
to_remove = []
for name in overview["title"]:
    if "Undercard" in word_tokenize(name):
        to_remove.append(count)
    count+=1

overview = overview.drop(to_remove)

overview = overview.reset_index(drop = True)

In [5]:
# clean each column and remove ones we don't want anymore

# "date" column

for p in overview["date"]:
    
    x = p.replace("\n","")
    if x[0] == "1":
        new_date = x[4:]
    else:
        new_date = x
        
    year = new_date.split(",")[1]
    month_day = new_date.split(",")[0]
    month = month_day.split(" ")[0]
    day = month_day.split(" ")[1]
    
    if len(day) == 1:
        str1 = " 0" + day
        final_returned = month + str1 + "," + year
        overview = overview.replace(p, final_returned)
    else:
        overview = overview.replace(p, new_date)

# "type" column

count = 0
new_type = []
for x in overview["type"]:
    if "Forum" in word_tokenize(overview["title"][count]):
        if word_tokenize(overview["title"][count])[0] == "Democratic":
            new_type.append("D")
        if word_tokenize(overview["title"][count])[0] == "Republican":
            new_type.append("R")
    
    else:
        if x == "Primary-D (democratic)":
            new_type.append("D")
        if x == "Primary-R (republican)":
            new_type.append("R")
        if x == "General":
            new_type.append("G") 
    count+=1

overview["type"] = new_type
    
# "title" column

for x in overview["title"]:
    t = x.split(",")
    title = t[0].replace('"', "")
    overview = overview.replace(x, title)   
    
# "moderator" column

for x in overview["moderator"]:
    
    a = ((x[2:])[:-2]).split(",")
    dist = []
    
    for i in a:
        word=i.split("(")[0]
        name = word.split(" ")
        name = list(filter(None, name))
        last = name[-1:]
        for i in last:
            dist.append(i)
    overview = overview.replace(x, str(dist))

# attributing file_names to each debate
file_name = []
for i in range(0, len(overview.index)):
    name = overview["date"][i].replace("\n","") + ("-") + overview["title"][i].replace("\n","")
    name = name.replace(",","")
    file_name.append(name)

overview["file_name"] = file_name
overview = overview.rename(columns = {"file_name": "debate_name"})

# drop date, title
overview = overview.drop(["title"], axis=1)

In [6]:
# asjustments to moderator column

count = 0
for x in overview["moderator"]:
    
    # changed from being a string to a list string
    new = re.sub("]|'", "", x).strip("[").strip(".").strip("-").replace(" ","").replace('"',"").split(",")
    
    # update field
    overview["moderator"][count] = new
    
    count+=1
    
news_sources = ["CNN","PBS","host","WMUR","MSNBC","CNBC","News","editor"]
news_sources = list(set(news_sources + [x.lower() for x in news_sources] + [x.upper() for x in news_sources] + [x.capitalize() for x in news_sources]))

count = 0
for row in overview["moderator"]:
    new_row = []
    for word in row:
        if word not in news_sources:
            word_C = word.capitalize()
            new_row.append(word_C)
    
    # update field
    overview["moderator"][count] = new_row
    
    count+=1

In [7]:
# new column to show election year

overview["election_year"] = np.nan

count = 0
for row in overview['year']:
    if (row%2) == 0:
        overview['election_year'][count] = row
    else:
        overview['election_year'][count] = row+1
        election_year = row+1
    count+=1

In [8]:
# new column for debate coder

overview["coder"] = np.nan

coder = []

for i in range(len(overview)):
    t = str(overview["type"][i])
    y = str(overview["year"][i])
    m = overview["date"][i][:3]
    d = str(overview["date"][i][-8:-6])
    full = t+y+str(m.lower())+d
    coder.append(full)
    
overview["coder"] = coder

overview = overview.drop(columns = ["year"])

In [10]:
overview.head()

,date,type,moderator,debate_name,election_year,coder
0,"October 21, 1960",G,"[Howe, Edwards, Vanocur]",October 21 1960-Presidential Debate in New York,1960.0,G1960oct21
1,"October 22, 1976",G,"[Walters, Nelson, Kraft, Maynard]",October 22 1976-Presidential Campaign Debate i...,1976.0,G1976oct22
2,"October 13, 1988",G,"[Shaw, Compton, Warner, Mitchell]",October 13 1988-Presidential Debate in Los Ang...,1988.0,G1988oct13
3,"October 19, 1992",G,[Lehrer],October 19 1992-Presidential Debate in East La...,1992.0,G1992oct19
4,"October 16, 1996",G,[Lehrer],October 16 1996-Presidential Debate in San Diego,1996.0,G1996oct16


In [11]:
# output cleaned overview file
overview.to_csv("debate_data/data_overviews/overview_1.csv",index=False)